# Run repetition code memory experiment on IBM

In [1]:
import os

import numpy as np
import pickle

import qiskit
import qiskit_ibm_runtime

## Set experiment parameters

In [2]:
depths = np.arange(10, 101, 10)
nvals = [3, 5, 7, 9, 11, 13, 15]
nqubits = nvals[-1]
physical_qubits = [113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 112]
assert len(physical_qubits) == nqubits
nshots = 10_000

In [5]:
service = qiskit_ibm_runtime.QiskitRuntimeService()

# computer = qiskit_ibm_runtime.fake_provider.FakeSherbrooke()
computer = service.backend("ibm_sherbrooke")
sampler = qiskit_ibm_runtime.SamplerV2(computer)

In [ ]:
batch = []
for depth in depths:
    print(f"Status: depth = {depth}")
    qreg = qiskit.QuantumRegister(nqubits)
    circuit = qiskit.QuantumCircuit(qreg)

    # circuit.delay(duration=100, unit="dt")
    for _ in range(depth):
        circuit.x(qreg)
    circuit.measure_all()

    compiled = qiskit.transpile(
        circuit, 
        backend=computer,
        initial_layout=physical_qubits,
        # routing_method="sabre",
        # scheduling_method="asap",
        optimization_level=0,
    )
    batch.append(compiled)

batch[-1].draw(fold=-1, idle_wires=False)

In [7]:
job = sampler.run(
    batch,
    shots=nshots,
)
# job = service.job("ctwgnt7seb10008x520g")  # Get results from a Job ID.

In [ ]:
job.status()

## Save data

In [ ]:
dir = job.job_id()

In [11]:
os.mkdir(dir)

In [20]:
with open(f"{dir}/{dir}.pkl", "wb") as f:
    pickle.dump(job.result().to_dict(), f)

with open(f"{dir}/nqubits.txt", "w") as f:
    f.write(str(nqubits))

with open(f"{dir}/nshots.txt", "w") as f:
    f.write(str(nshots))


np.savetxt(f"{dir}/depths.txt", depths)
np.savetxt(f"{dir}/nvals.txt", nvals)
np.savetxt(f"{dir}/physical_qubits.txt", physical_qubits)